# Indexing MDFs with Pony ORM.
## MySQL

SQLlite is a good tool for development, but it does not scale well.

In addition to SQLLite PonyORM supports PostgreSQL, MySQL, and Oracle.

This example picks up from the SQLLite example but adds MySQL support.

Using a MySQL (or PostreSQL/Oracle) backend will allow for distributed processing on any node that can access the s3 buckets.

In [2]:
import os

import pony.orm
from pony.orm.core import EntityMeta
from datetime import datetime

pony.orm.set_sql_debug(False)

db = pony.orm.Database()

db.bind(
    provider='mysql',
    host='mysql',
    user='mdf_indexer_user',
    passwd='mdf_indexer_pass',
    db='mdf_database',
)

In [3]:
import asammdf
# For Local Indexing.
class MDF(db.Entity):
    """MDF ORM Entity Fancy"""
    # Filesystem Bits.
    key = pony.orm.Required(str, unique=True,)
    last_modified = pony.orm.Optional(datetime, volatile=True)
    etag = pony.orm.Optional(str,)
    size = pony.orm.Optional(int,)
    size_mb = pony.orm.Optional(float,)
    storage_class = pony.orm.Optional(str,)
    type = pony.orm.Optional(str,)
    name = pony.orm.Optional(str,)
    
    # Pre-calculated bits.
    basename = pony.orm.Optional(str,)
    product = pony.orm.Optional(str,)
    company = pony.orm.Optional(str,)

    # ASAM MDF Bits.
    version = pony.orm.Optional(str,)
    channels = pony.orm.Set("Channel",)
    
    
    # Basename.
    basename = pony.orm.Optional(str,)
    channels = pony.orm.Set("Channel",)
    
    # Metadata
    product = pony.orm.Optional(str,)
    company = pony.orm.Optional(str,)
    
    def __repr__(self):
        return f"MDF<{self.id},{self.product},{self.company},Ch:{len(self.channels)}>"

class Channel(db.Entity):
    """Channel entity to represent a 
    
    """
    name = pony.orm.Required(str, unique=True,)
    mdfs = pony.orm.Set("MDF",)
    
    def __repr__(self):
        return f"Channel<{self.id},{self.name}>"

def upsert(cls, get, set=None):
    """
    Interacting with Pony entities.

    :param cls: The actual entity class
    :param get: Identify the object (e.g. row) with this dictionary
    :param set: Additional fields to set if ```get``` returns nothing.
    :return:
    """
    # does the object exist
    assert isinstance(cls, EntityMeta), f"{cls} is not a database entity"

    # if no set dictionary has been specified
    set = set or {}
    db.flush()
    if not cls.exists(**get):
        # make new object
        return cls(**set, **get)
    else:
        # get the existing object
        obj = cls.get(**get)
        for key, value in set.items():
            obj.__setattr__(key, value)
        return obj
    

db.generate_mapping(create_tables=True)

In [5]:
MDF.select().first()

MDF<1,Excavator,ABMøøse,Ch:13>

In [9]:
Channel.select().first()

Channel<1,time>